In [1]:
import h5py
import numpy as np

In [18]:
with h5py.File('data/deposition.h5') as ipt_deposition:
    Response_Matrix = ipt_deposition['Matrix'][...]

Response_Matrix_old = Response_Matrix       # 这是一个用于检验reshape正确性所用的语句

# Response_Matrix 
# 最终索引：(theta, phi, Det, Eng_in, Engout)
# 大小：    (7, 12, 4, 40, 100)
Response_Matrix = Response_Matrix.reshape(40, 7, 12, 4, 100)
Response_Matrix = Response_Matrix.swapaxes(0, 1).swapaxes(1, 2).swapaxes(2, 3)
Response_Matrix.shape

/tmp/ipykernel_3642/510504703.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File('data/deposition.h5') as ipt_deposition:


(7, 12, 4, 40, 100)

In [55]:
# 用于检验矩阵分割的正确性的函数
# 最终期望结果为True(1)
# 100,000 次循环DM0U本地用时2.4s

def get_Matrix_id(id_engin, id_theta, id_phi):
    id = id_phi + id_theta * 12 + id_engin * 7 * 12
    return id

_flag = True
for _i in range(100000):
    id_engin = np.random.randint(40)
    id_theta = np.random.randint(7)
    id_phi = np.random.randint(12)
    _Bool = (Response_Matrix_old[get_Matrix_id(id_engin, id_theta, id_phi)] == \
             Response_Matrix[id_theta, id_phi, :, id_engin])
    _flag *= np.prod(_Bool)

    if _flag == False:
        break

print(_flag)


1


In [85]:
id_theta = 0
id_det = 0
print(Response_Matrix[id_theta, :, id_det, 0])
print(Response_Matrix[id_theta, :, id_det, 0].sum())

[[171   0   0 ...   0   0   0]
 [159   0   0 ...   0   0   0]
 [170   0   0 ...   0   0   0]
 ...
 [191   0   0 ...   0   0   0]
 [180   0   0 ...   0   0   0]
 [175   0   0 ...   0   0   0]]
2115


In [114]:
def get_Response_Matrix(theta=2, phi=3):
    '''
    双线性插值，计算theta, phi方向入射的响应矩阵
        TODO: 或许应该改为接受一维numpy列表的theta,phi？
        intput:
            theta       入射的顶角, float
            phi         入射的旋转角,float
        output:
            大小为(det, Engin, Engout)的numpy数组，表示theta, phi方向的响应矩阵
    '''
    global Response_Matrix

    id_phi = phi / 2 / np.pi * 12
    id_phi_inf = int(np.floor(id_phi))
    id_phi_sup = (id_phi_inf + 1) % 12

    delta_id_phi = id_phi - id_phi_inf


    id_theta = theta / np.pi * 6
    id_theta_inf = int(np.floor(id_theta))
    id_theta_sup = (id_theta_inf + 1) % 6

    delta_id_theta = id_theta - id_theta_inf

    return \
    (1 - delta_id_phi) * \
        (Response_Matrix[id_theta_inf, id_phi_inf] * (1 - delta_id_theta) + \
        Response_Matrix[id_theta_sup, id_phi_inf] * delta_id_theta) + \
    delta_id_phi * \
        (Response_Matrix[id_theta_inf, id_phi_sup] * (1 - delta_id_theta) + \
        Response_Matrix[id_theta_sup, id_phi_sup] * delta_id_theta)

In [117]:
# 双线性算法正确性的简单检验

theta_unit = np.pi / 6
phi_unit = np.pi * 2 / 12
a = get_Response_Matrix(theta_unit, phi_unit / 2)
b = (Response_Matrix[1, 0] + Response_Matrix[1, 1]) / 2

np.prod(a == b)